Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

---

In [1]:
from random import randrange
import numpy as np
from sklearn.metrics import mean_squared_error, log_loss
from sklearn.datasets import load_breast_cancer, load_diabetes
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.preprocessing import normalize


def grad_check_sparse(f, x, analytic_grad, num_checks=10, h=1e-5, error=1e-9):
    """
    sample a few random elements and only return numerical
    in this dimensions
    """

    for i in range(num_checks):
        ix = tuple([randrange(m) for m in x.shape])

        oldval = x[ix]
        x[ix] = oldval + h  # increment by h
        fxph = f(x)  # evaluate f(x + h)
        x[ix] = oldval - h  # increment by h
        fxmh = f(x)  # evaluate f(x - h)
        x[ix] = oldval  # reset

        grad_numerical = (fxph - fxmh) / (2 * h)
        grad_analytic = analytic_grad[ix]
        rel_error = abs(grad_numerical - grad_analytic) / (
            abs(grad_numerical) + abs(grad_analytic)
        )
        print(
            "numerical: %f analytic: %f, relative error: %e"
            % (grad_numerical, grad_analytic, rel_error)
        )
        assert rel_error < error

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [2]:
data = load_diabetes()
X, y = data.data, data.target

In [3]:
def mse_loss_vectorized(w, b, X, y):
    """
    MSE loss function WITHOUT FOR LOOPs , NO REGULARIZATION
    
    Returns a tuple of:
    - loss 
    - gradient with respect to weights w
    - gradient with respect to bias b
    """
    loss = 0.0
    dw = np.zeros_like(w)
    
    # YOUR CODE HERE
    # raise NotImplementedError()
    y_pred = X.dot(w) + b
    
    # loss 
    loss = (1.0 / X.shape[0]) * ((y - y_pred) ** 2).sum()
    
    # gradient with respect to weights
    dw = - (2.0 / X.shape[0]) * X.T.dot(y - y_pred)
    
    # gradient with respect to bias
    db = - (2.0 / X.shape[0]) * ((y - y_pred)).sum()
    
    return loss, dw, np.array(db).reshape(1,)

https://rpubs.com/Xiaocai/379592
https://www.kaggle.com/code/residentmario/soft-thresholding-with-lasso-regression/notebook

In [4]:
def soft_threshold(x, delta):
    # YOUR CODE HERE
    # raise NotImplementedError()
    if x > delta:
        return x - delta
    elif x < (- delta):
        return x + delta
    else:
        return 0

# Lasso Subgradient Descent

In [5]:
def l1_subgradient(w):
    """
    Subgradient of the L1 loss
    """
    dw = np.zeros_like(w)
    # YOUR CODE HERE
    # raise NotImplementedError()
    
    dw = np.sign(w)
    return dw

def lasso_subgradient_mse_loss_vectorized(w, b, X, y, alpha):
    """
    MSE loss function adding the subgradient for w
    """
    loss, dw, db = mse_loss_vectorized(w, b, X, y)
    # Add the subgradient to dw
    # YOUR CODE HERE
    # raise NotImplementedError()
    
    dw += alpha * l1_subgradient(w)
    return loss, dw, db

In [6]:
class LassolSubgradientDescent():
    def __init__(self,  alpha=0.1):
        self.w = None
        self.b = None
        self.alpha = alpha

    def train(self, X, y, learning_rate=1e-3, num_iters=100, batch_size=200, verbose=False):
        N, d = X.shape
        
        if self.w is None: # Initialization
            self.w = 0.001 * np.random.randn(d)
            self.b = 0.0

        # Run stochastic gradient descent to optimize w
        
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
                                                               
            # Sample batch_size elements in X_batch and y_batch
            # X_batch shape is  (batch_size, d) and y_batch shape is (batch_size,)                                                                                          
            # Hint: Use np.random.choice to generate indices
            # YOUR CODE HERE
            # raise NotImplementedError()
            choices = np.random.choice(N, batch_size, replace=False)
            
            X_batch = X[choices, :]
            y_batch = y[choices]
            
            # evaluate loss and gradient
            loss, dw, db = self.loss(X_batch, y_batch)

            # perform parameter update                                                                
            # Update the weights w using the gradient and the learning rate.  
            # YOUR CODE HERE
            # raise NotImplementedError()
            self.w -= learning_rate * dw
            self.b -= learning_rate * db
            
            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))
    
    
    def predict(self, X):
        # YOUR CODE HERE
        # raise NotImplementedError()
        y_pred = X.dot(self.w) + self.b
        return y_pred

    def loss(self, X_batch, y_batch):
        return lasso_subgradient_mse_loss_vectorized(self.w, self.b, X_batch, y_batch, self.alpha)

In [7]:
model = LassolSubgradientDescent(alpha=0.1)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse = mean_squared_error(pred, y)

sk_model = Lasso(alpha=0.1, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 26823.781012
iteration 10000 / 200000: loss 3513.986187
iteration 20000 / 200000: loss 3244.827846
iteration 30000 / 200000: loss 2901.080772
iteration 40000 / 200000: loss 2874.278726
iteration 50000 / 200000: loss 2746.304920
iteration 60000 / 200000: loss 2975.101279
iteration 70000 / 200000: loss 2464.897137
iteration 80000 / 200000: loss 3165.359047
iteration 90000 / 200000: loss 2800.433954
iteration 100000 / 200000: loss 3236.405989
iteration 110000 / 200000: loss 2497.484149
iteration 120000 / 200000: loss 2989.652798
iteration 130000 / 200000: loss 2986.302230
iteration 140000 / 200000: loss 3132.918971
iteration 150000 / 200000: loss 2783.598636
iteration 160000 / 200000: loss 3111.435969
iteration 170000 / 200000: loss 2724.482056
iteration 180000 / 200000: loss 2847.456401
iteration 190000 / 200000: loss 2778.634817
MSE scikit-learn: 2912.521795117546
MSE Coordinate descent model : 2888.936905005078


In [8]:
model = LassolSubgradientDescent(alpha=2)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse = mean_squared_error(pred, y)

sk_model = Lasso(alpha=2, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 28538.933103
iteration 10000 / 200000: loss 4559.724505
iteration 20000 / 200000: loss 3602.320496
iteration 30000 / 200000: loss 3724.699878
iteration 40000 / 200000: loss 3931.723151
iteration 50000 / 200000: loss 4094.531449
iteration 60000 / 200000: loss 3674.446541
iteration 70000 / 200000: loss 4011.911913
iteration 80000 / 200000: loss 3368.886326
iteration 90000 / 200000: loss 3476.223895
iteration 100000 / 200000: loss 3972.318205
iteration 110000 / 200000: loss 4027.532532
iteration 120000 / 200000: loss 3480.428465
iteration 130000 / 200000: loss 3638.098722
iteration 140000 / 200000: loss 3636.810858
iteration 150000 / 200000: loss 3991.893475
iteration 160000 / 200000: loss 3457.385396
iteration 170000 / 200000: loss 3864.188688
iteration 180000 / 200000: loss 3698.949016
iteration 190000 / 200000: loss 4065.737345
MSE scikit-learn: 5650.287416333697
MSE Coordinate descent model : 3809.400527029728


# Lasso Proximal Gradient Descent (iterative soft thresholding algorithm or ISTA)

In [9]:
class LassoProximalGradientDescent():
    def __init__(self,  alpha=0.1):
        self.w = None
        self.b = None
        self.alpha = alpha

    def train(self, X, y, learning_rate=1e-3, num_iters=100, batch_size=200, verbose=False):
        N, d = X.shape
        
        if self.w is None: # Initialization
            self.w = 0.001 * np.random.randn(d)
            self.b = 0.0

        # Run stochastic gradient descent to optimize w
        
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
                                                               
            # Sample batch_size elements in X_batch and y_batch
            # X_batch shape is  (batch_size, d) and y_batch shape is (batch_size,)                                                                                          
            # Hint: Use np.random.choice to generate indices
            # YOUR CODE HERE
            # raise NotImplementedError()
            choices = np.random.choice(N, batch_size, replace=False)
            
            X_batch = X[choices, :]
            y_batch = y[choices]
            
            # evaluate loss and gradient
            loss, dw, db = self.loss(X_batch, y_batch)

            # perform parameter update                                                                
            # Update the weights w using the gradient and the learning rate.  
            # PROJECT THE GRADIENT FOR w USING soft_threshold
            # YOUR CODE HERE
            # raise NotImplementedError()
            
            for i in range(d):
                self.w[i] = soft_threshold(self.w[i] - learning_rate * dw[i], self.alpha * learning_rate)

            self.b -= learning_rate * db
            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))

    def predict(self, X):
        # YOUR CODE HERE
        # raise NotImplementedError()
        y_pred = X.dot(self.w) + self.b
        return y_pred

    def loss(self, X_batch, y_batch):
        return mse_loss_vectorized(self.w, self.b, X_batch, y_batch)

In [10]:
model = LassoProximalGradientDescent(alpha=0.1)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=0.1, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 30476.981964
iteration 10000 / 200000: loss 3222.597134
iteration 20000 / 200000: loss 3174.351654
iteration 30000 / 200000: loss 3138.817671
iteration 40000 / 200000: loss 2743.299522
iteration 50000 / 200000: loss 2701.215613
iteration 60000 / 200000: loss 2712.943244
iteration 70000 / 200000: loss 2996.698096
iteration 80000 / 200000: loss 2940.145579
iteration 90000 / 200000: loss 2506.953224
iteration 100000 / 200000: loss 2625.918112
iteration 110000 / 200000: loss 3006.592098
iteration 120000 / 200000: loss 3206.959709
iteration 130000 / 200000: loss 2676.412980
iteration 140000 / 200000: loss 2831.942673
iteration 150000 / 200000: loss 2766.138862
iteration 160000 / 200000: loss 2556.010907
iteration 170000 / 200000: loss 2987.070923
iteration 180000 / 200000: loss 2869.074655
iteration 190000 / 200000: loss 2800.779310
MSE scikit-learn: 2912.521795117546
MSE Coordinate descent model : 2888.677615272093


In [11]:
model = LassoProximalGradientDescent(alpha=2)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=2, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 29601.318125
iteration 10000 / 200000: loss 4786.442964
iteration 20000 / 200000: loss 3898.549557
iteration 30000 / 200000: loss 3715.769432
iteration 40000 / 200000: loss 3983.054218
iteration 50000 / 200000: loss 3925.521334
iteration 60000 / 200000: loss 3762.723420
iteration 70000 / 200000: loss 3533.433531
iteration 80000 / 200000: loss 3845.723410
iteration 90000 / 200000: loss 3632.757456
iteration 100000 / 200000: loss 3948.627935
iteration 110000 / 200000: loss 3863.351670
iteration 120000 / 200000: loss 3712.963624
iteration 130000 / 200000: loss 3764.964743
iteration 140000 / 200000: loss 3589.052847
iteration 150000 / 200000: loss 4153.753909
iteration 160000 / 200000: loss 3710.887550
iteration 170000 / 200000: loss 4080.304439
iteration 180000 / 200000: loss 3647.606875
iteration 190000 / 200000: loss 3750.152881
MSE scikit-learn: 5650.287416333697
MSE Coordinate descent model : 3812.5934334191165


# Lasso Projected Gradient Descent

In [12]:
class LassoProjectedGradientDescent():
    def __init__(self,  alpha=0.1):
        self.w_p = None
        self.w_n = None
        self.b = None
        self.alpha = alpha

    def train(self, X, y, learning_rate=1e-3, num_iters=100, batch_size=200, verbose=False):
        N, d = X.shape
        
        if self.w_p is None: # Initialization
            self.w_p = 0.001 * np.random.randn(d)
            self.w_n = 0.001 * np.random.randn(d)
            self.b = 0.0

        # Run stochastic gradient descent to optimize w
        
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
                                                               
            # Sample batch_size elements in X_batch and y_batch
            # X_batch shape is  (batch_size, d) and y_batch shape is (batch_size,)                                                                                          
            # Hint: Use np.random.choice to generate indices
            # YOUR CODE HERE
            # raise NotImplementedError()
            choices = np.random.choice(N, batch_size, replace=False)
            
            X_batch = X[choices, :]
            y_batch = y[choices]
            
            # evaluate loss and gradient
            loss, dw, db = self.loss(X_batch, y_batch)

            # perform parameter update                                                                
            # Update the weights w using the gradient and the learning rate.  
            # Project for w_p and w_n
            # YOUR CODE HERE
            # raise NotImplementedError()
            
            w = self.w_p - self.w_n
            grad = - X_batch.T.dot(y_batch - (X_batch.dot(w)))
    
            grad_wn = -grad + self.alpha * l1_subgradient(w)
            grad_wp = grad + self.alpha * l1_subgradient(w)
            
            self.w_p -= learning_rate * grad_wp
            self.w_n -= learning_rate * grad_wn
            
            self.b -= learning_rate * db
            
            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))

    def predict(self, X):
        # YOUR CODE HERE
        # raise NotImplementedError()
        y_pred = X.dot(self.w_p - self.w_n) + self.b
        return y_pred

    def loss(self, X_batch, y_batch):
        # YOUR CODE HERE
        # raise NotImplementedError()
        return mse_loss_vectorized((self.w_p-self.w_n), self.b, X_batch, y_batch)

In [13]:
model = LassoProjectedGradientDescent(alpha=0.1)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=0.1, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 25737.944138
iteration 10000 / 200000: loss 2673.887555
iteration 20000 / 200000: loss 3112.451082
iteration 30000 / 200000: loss 2845.420806
iteration 40000 / 200000: loss 2795.368397
iteration 50000 / 200000: loss 2883.648747
iteration 60000 / 200000: loss 3060.706716
iteration 70000 / 200000: loss 3036.010986
iteration 80000 / 200000: loss 2934.481433
iteration 90000 / 200000: loss 2773.768284
iteration 100000 / 200000: loss 2574.896259
iteration 110000 / 200000: loss 2923.989181
iteration 120000 / 200000: loss 2788.028719
iteration 130000 / 200000: loss 2923.733261
iteration 140000 / 200000: loss 2799.588472
iteration 150000 / 200000: loss 2762.835678
iteration 160000 / 200000: loss 2638.381251
iteration 170000 / 200000: loss 2753.412406
iteration 180000 / 200000: loss 2791.319781
iteration 190000 / 200000: loss 2783.290489
MSE scikit-learn: 2912.521795117546
MSE Coordinate descent model : 2864.9129740590306


In [14]:
model = LassoProjectedGradientDescent(alpha=2)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=2, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 29388.183288
iteration 10000 / 200000: loss 2941.226199
iteration 20000 / 200000: loss 2733.720440
iteration 30000 / 200000: loss 2383.118418
iteration 40000 / 200000: loss 2817.790370
iteration 50000 / 200000: loss 3097.643347
iteration 60000 / 200000: loss 2927.393473
iteration 70000 / 200000: loss 2864.936737
iteration 80000 / 200000: loss 2853.214005
iteration 90000 / 200000: loss 2637.645933
iteration 100000 / 200000: loss 2880.526350
iteration 110000 / 200000: loss 3004.912060
iteration 120000 / 200000: loss 2659.252430
iteration 130000 / 200000: loss 3050.710026
iteration 140000 / 200000: loss 3161.979292
iteration 150000 / 200000: loss 2873.458317
iteration 160000 / 200000: loss 2701.615498
iteration 170000 / 200000: loss 2748.039509
iteration 180000 / 200000: loss 2668.728784
iteration 190000 / 200000: loss 2853.048170
MSE scikit-learn: 5650.287416333697
MSE Coordinate descent model : 2861.5409522955647


# Lasso Coordinate Descent (no intercept)

In [15]:
class LassoCoordinateDescent():
    def __init__(self, alpha=0.1):
        self.w = None
        self.alpha = alpha

    def train(self, X, y, num_iters=1000):
        N, d = X.shape
        
        # YOUR CODE HERE
        # raise NotImplementedError()
        # code from https://xavierbourretsicotte.github.io/lasso_implementation.html
        
        #normalizing X in case it was not done before
        X = X / (np.linalg.norm(X, axis = 0)) 
        
        # Initialization
        self.w = np.ones((d,1))
            
        soft = np.vectorize(soft_threshold)
        
        #Looping until max number of iterations
        for i in range(num_iters): 

            #Looping through each coordinate
            for j in range(d):

                #Vectorized implementation
                X_j = X[:,j].reshape(-1,1)
                y_pred = (X @ self.w).reshape(-1)
                rho = X_j.T @ (y - y_pred.squeeze() + (self.w[j] * X_j).squeeze())
                # no fit_intercept
                self.w[j] = soft_threshold(rho, self.alpha)
                
        self.w.flatten()

    def predict(self, X): 
        # YOUR CODE HERE
        # raise NotImplementedError()
        y_pred = X.dot(self.w)
        return y_pred

In [16]:
model = LassoCoordinateDescent(alpha=0.1)
model.train(X, y)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=0.1, fit_intercept=False)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

MSE scikit-learn: 26057.118798659812
MSE Coordinate descent model : 26004.29770934135


In [17]:
model = LassoCoordinateDescent(alpha=2)
model.train(X, y)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=2, fit_intercept=False)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

MSE scikit-learn: 28794.88441987604
MSE Coordinate descent model : 26006.416566913224
